# Notebook to correct quenching from a glider dataset

The idea is to leverage the number of observation in a glider dataset to achieve a quenching correction using the night time profile as our training dataset of "unquenched profiles".

1) Binary classification of quenched profiles. The mid-day profiles will be labelled as quenched and mid-night profiles will be labeled as unquenched to predict if dusk and dawn profiles are quenched. 
2) Second binary classification to predict the quenched layer depth based on F/bbp values. 
3) Predict a F/bbp factor based on bbp, T, S, depth, Doy. Training is realised on the "unquenched database" and new prediction is made on quenched layers. 
4) Apply the ratio predicted before on the quench layer

## 1. Building the dataset 

Open the gliders dataset. I want to end up with one tidy table composed of profile id, doy, time, depth, T, S, F, bbp.
I will further derive a "day_period" variable that will correspond to mid-day, dusk, mid-night or dawn. TBD what time lag should be before/after sunset and sunset to consider mid-day or mid-night.

In [ ]:
import xarray as xr
import polars as pl
import glob

# Get list of NetCDF files
nc_files = glob.glob("C:/Users/flapet/OneDrive - NOC/Documents/IDAPro/lib/db_building/data/glider/nc_profiled/*.nc")

# Read and concatenate datasets
for file in nc_files:
    ds = xr.open_dataset(file, decode_times=True)
    glider_name = ds.attrs.get("trajectory").split("_")[0]
    # Select variables of interest
    vars_of_interest = ["TIME", "DEPTH", "TEMP", "CHLA", "BBP700", "ABS_SALINITY", "PROFILE_NUMBER", "LATITUDE", "LONGITUDE"]
    ds_sel = ds[vars_of_interest]
    # Convert to pandas DataFrame, then to polars DataFrame
    df_pd = ds_sel.to_dataframe().reset_index()
    df_pl = pl.from_pandas(df_pd)

    #Use only profiling data (-1 is surfacing behavior)
    df_pl = df_pl.filter(pl.col("PROFILE_NUMBER") > 0)

    df_pl = df_pl.with_columns((pl.col("PROFILE_NUMBER").cast(pl.String) + "_" + glider_name).alias("profile_id"))
    print(f"{file} processed, shape: {df_pl.shape}")

    # Concatenate DataFrames
    if 'df_concat' in locals():
        df_concat = pl.concat([df_concat, df_pl])
    else:
        df_concat = df_pl

df_concat

In [3]:
# Interpolate DEPTH, TEMP, and ABS_SALINITY per profile_id along TIME
df_concat = (
    df_concat
    .sort(["profile_id", "TIME"])
    .with_columns([
        pl.col("DEPTH").interpolate().over("profile_id").alias("DEPTH_interp"),
        pl.col("TEMP").interpolate().over("profile_id").alias("TEMP_interp"),
        pl.col("ABS_SALINITY").interpolate().over("profile_id").alias("ABS_SALINITY_interp"),
    ])
)

In [4]:
# Count rows where CHLA and BBP700 are not null, but at least one of the interp columns is null
count = (
    df_concat
    .filter(
        (pl.col("CHLA").is_not_null()) &
        (pl.col("BBP700").is_not_null()) &
        (
            pl.col("DEPTH_interp").is_null() |
            pl.col("TEMP_interp").is_null() |
            pl.col("ABS_SALINITY_interp").is_null()
        )
    )
    .height
)
print(f"Number of rows with CHLA and BBP700 but missing one interp variable: {count}")

Number of rows with CHLA and BBP700 but missing one interp variable: 8451


In [5]:
# Filter out rows with NA in required columns and drop original columns
df_filtered = (
    df_concat
    .filter(
        pl.col("CHLA").is_not_null() &
        pl.col("BBP700").is_not_null() &
        pl.col("DEPTH_interp").is_not_null() &
        pl.col("TEMP_interp").is_not_null() &
        pl.col("ABS_SALINITY_interp").is_not_null()
    )
    .drop(["DEPTH", "TEMP", "ABS_SALINITY"])
    .rename({
        "DEPTH_interp": "DEPTH",
        "TEMP_interp": "TEMP",
        "ABS_SALINITY_interp": "ABS_SALINITY"
    })
)
df_filtered

N_MEASUREMENTS,TIME,CHLA,BBP700,PROFILE_NUMBER,LATITUDE,LONGITUDE,profile_id,DEPTH,TEMP,ABS_SALINITY
i64,datetime[ns],f32,f32,f64,f32,f32,str,f64,f32,f64
326,2024-05-28 12:59:32.235504896,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.18995,8.951067,0.000369
330,2024-05-28 12:59:42.302062848,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.183345,8.94232,0.000369
334,2024-05-28 12:59:52.421722112,18.9946,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.186647,8.9334,0.00037
337,2024-05-28 13:00:02.500366336,18.987301,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.18995,8.9259,0.00037
340,2024-05-28 13:00:12.561890304,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.185822,8.929325,0.00037
…,…,…,…,…,…,…,…,…,…,…
1374725,2024-08-19 17:30:46.156890112,0.9782,0.000329,999.0,62.47134,-16.046728,"""999.0_Nelson""",-5.793653,12.247167,35.358
1374729,2024-08-19 17:30:56.291320320,0.8979,0.000336,999.0,62.471344,-16.046844,"""999.0_Nelson""",-4.714164,12.247167,35.357858
1374731,2024-08-19 17:31:06.436065280,0.9198,0.000321,999.0,62.471352,-16.046959,"""999.0_Nelson""",-4.179369,12.246634,35.356385


In [9]:
# Compute unique dates with average lat/lon for each date
date_latlon = (
    df_filtered
    .groupby("DATE")
    .agg([
        pl.col("LATITUDE").mean().alias("LAT_MEAN"),
        pl.col("LONGITUDE").mean().alias("LON_MEAN")
    ])
)

# Function to compute sunrise and sunset for a date/lat/lon
def compute_sun_times_row(row):
    date = row["DATE"]
    lat = row["LAT_MEAN"]
    lon = row["LON_MEAN"]
    try:
        location = LocationInfo(latitude=lat, longitude=lon)
        s = sun(location.observer, date=date, tzinfo=pytz.UTC)
        return s["sunrise"].time(), s["sunset"].time()
    except Exception:
        return None, None

# Apply function to each row and create two new columns
date_latlon = date_latlon.with_columns([
    pl.struct(["DATE", "LAT_MEAN", "LON_MEAN"]).apply(
        lambda x: compute_sun_times_row(x)[0]
    ).alias("SUNRISE"),
    pl.struct(["DATE", "LAT_MEAN", "LON_MEAN"]).apply(
        lambda x: compute_sun_times_row(x)[1]
    ).alias("SUNSET"),
])

# Join back to df_filtered on DATE
df_filtered = df_filtered.join(date_latlon.select(["DATE", "SUNRISE", "SUNSET"]), on="DATE", how="left")

df_filtered


C:\Users\flapet\AppData\Local\Temp\ipykernel_33452\452025040.py:4: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby("DATE")
C:\Users\flapet\AppData\Local\Temp\ipykernel_33452\452025040.py:25: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.struct(["DATE", "LAT_MEAN", "LON_MEAN"]).apply(
C:\Users\flapet\AppData\Local\Temp\ipykernel_33452\452025040.py:28: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.struct(["DATE", "LAT_MEAN", "LON_MEAN"]).apply(


N_MEASUREMENTS,TIME,CHLA,BBP700,PROFILE_NUMBER,LATITUDE,LONGITUDE,profile_id,DEPTH,TEMP,ABS_SALINITY,DATE,DOY,TIME_ONLY,SUNRISE,SUNSET
i64,datetime[ns],f32,f32,f64,f32,f32,str,f64,f32,f64,date,i16,time,time,time
326,2024-05-28 12:59:32.235504896,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.18995,8.951067,0.000369,2024-05-28,149,12:59:32.235504896,04:31:29.290020,22:36:14.805490
330,2024-05-28 12:59:42.302062848,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.183345,8.94232,0.000369,2024-05-28,149,12:59:42.302062848,04:31:29.290020,22:36:14.805490
334,2024-05-28 12:59:52.421722112,18.9946,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.186647,8.9334,0.00037,2024-05-28,149,12:59:52.421722112,04:31:29.290020,22:36:14.805490
337,2024-05-28 13:00:02.500366336,18.987301,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.18995,8.9259,0.00037,2024-05-28,149,13:00:02.500366336,04:31:29.290020,22:36:14.805490
340,2024-05-28 13:00:12.561890304,18.9727,0.007497,1.0,60.000038,-24.001507,"""1.0_Cabot""",10.185822,8.929325,0.00037,2024-05-28,149,13:00:12.561890304,04:31:29.290020,22:36:14.805490
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1374725,2024-08-19 17:30:46.156890112,0.9782,0.000329,999.0,62.47134,-16.046728,"""999.0_Nelson""",-5.793653,12.247167,35.358,2024-08-19,232,17:30:46.156890112,05:18:08.059272,20:54:58.107853
1374729,2024-08-19 17:30:56.291320320,0.8979,0.000336,999.0,62.471344,-16.046844,"""999.0_Nelson""",-4.714164,12.247167,35.357858,2024-08-19,232,17:30:56.291320320,05:18:08.059272,20:54:58.107853
1374731,2024-08-19 17:31:06.436065280,0.9198,0.000321,999.0,62.471352,-16.046959,"""999.0_Nelson""",-4.179369,12.246634,35.356385,2024-08-19,232,17:31:06.436065280,05:18:08.059272,20:54:58.107853
